In [20]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from keras import models,Input,layers
from keras.applications.vgg16 import VGG16

In [21]:
x_train = np.loadtxt('input.csv',delimiter = ',')
y_train = np.loadtxt('labels.csv',delimiter = ',')
x_test = np.loadtxt('input_test.csv',delimiter = ',')
y_test = np.loadtxt('labels_test.csv', delimiter = ',')

x_train = x_train.reshape(len(x_train),100,100,3)
y_train = y_train.reshape(len(y_train),1)
x_test = x_test.reshape(len(x_test),100,100,3)
y_test = y_test.reshape(len(y_test),1)

In [22]:
input_shape = (100,100,3)
inputs = Input(shape = input_shape)

In [23]:
conv_base = VGG16(
    weights = 'imagenet',
    include_top = False,
    input_shape = (100,100,3)
)
conv_base.trainable = True
conv_base.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 100, 100, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 100, 100, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 50, 50, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 50, 50, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 50, 50, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 25, 25, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 25, 25, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 25, 25, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 25, 25, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 12, 12, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 12, 12, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 12, 12, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 3, 3, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

for layer in conv_base.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True


In [27]:
x = conv_base(inputs)
x = layers.Flatten()(x)

x = layers.Dense(64,activation = 'relu')(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

model = models.Model(inputs = inputs, outputs = outputs)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [28]:
model.fit(x_train,y_train,epochs = 10,validation_data = (x_test,y_test))

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 51s 806ms/step - accuracy: 0.5037 - loss: 8.9079 - val_accuracy: 0.7850 - val_loss: 0.8319
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 52s 821ms/step - accuracy: 0.8188 - loss: 0.5400 - val_accuracy: 0.8400 - val_loss: 0.3501
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 52s 822ms/step - accuracy: 0.9018 - loss: 0.2415 - val_accuracy: 0.8500 - val_loss: 0.3311
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 52s 822ms/step - accuracy: 0.9309 - loss: 0.1650 - val_accuracy: 0.8725 - val_loss: 0.2840
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 52s 822ms/step - accuracy: 0.9681 - loss: 0.0825 - val_accuracy: 0.8500 - val_loss: 0.4633
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 52s 823ms/step - accuracy: 0.9853 - loss: 0.0460 - val_accuracy: 0.8800 - val_loss: 0.3849
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 54s 864ms/step - accuracy: 0.9971 - loss: 0.0168 - val_accuracy: 0.8800 - val_loss: 0.4753
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 53s 843ms/step - accuracy: 0.9984 - loss: 0.0106 - val_accu

In [29]:
model.evaluate(x_test,y_test)

13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 533ms/step - accuracy: 0.9012 - loss: 0.4762


[0.47671613097190857, 0.8924999833106995]

In [ ]:
model.save('fine_tunin.keras'